In [1]:
import numpy as np
import os
import pandas as pd
import seaborn as sns
from pandas.plotting import register_matplotlib_converters
register_matplotlib_converters()
import datetime
from os.path import isfile, join
from sys import getsizeof
import glob

from random import *

from sklearn import preprocessing
from sklearn import metrics
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import ParameterGrid
from sklearn.model_selection import StratifiedKFold

from pickle import dump
import joblib

# from keras import backend as K
import tensorflow.keras as k
from tensorflow.keras import utils, models, layers, optimizers
from tensorflow.keras.wrappers.scikit_learn import KerasClassifier, KerasRegressor
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.models import load_model

import tensorflow as tf

In [3]:
# Load the model

model = load_model("./model/Optimal-Weights-008--71.51781.hdf5")

# Testing the Model

In [4]:
def get_prediction(data,model):
    """
    Predict the class of a given data point.
    """
    return model.predict(data)

In [5]:
# Load sample data to test
# load dataset
df = pd.read_csv("D:/Revi/Courses/TMLC/Projects/2. Deep Learning/4. DMSP Particle Precipitate/Dataset/AI_Ready_DMSP_Data.csv")

df_test = df.sample(5)

df_test.to_csv("D:/Revi/Courses/TMLC/Projects/2. Deep Learning/4. DMSP Particle Precipitate/Dataset/AI_Ready_DMSP_Test.csv")

In [6]:
from sklearn.preprocessing import RobustScaler

In [7]:
feature_cols = ['Bx_6hr','AL_6hr','vsw_6hr','psw_6hr','Bz_3hr','By_3hr','psw_1hr','By_45min','AL_45min','Bz_10min']

In [10]:
def PreProcessData(df):
    
    # Scale and Transform inputs
    X_val = df[feature_cols].copy(deep=True)
    y_val = df['ELE_TOTAL_ENERGY_FLUX'].copy(deep=True)
    scaler_X = RobustScaler()
    scaler_X = scaler_X.fit(X_val.values)
    X_val_scaled = scaler_X.transform(X_val.values)
    
    # Scale and Transform labels
    # The units of the energy flux are eV/cm2/s/ster
    #   --> To remove the 'steradian' multiply by PI (see: https://agupubs.onlinelibrary.wiley.com/doi/epdf/10.1002/2016JA023339)
    y_val = y_val * np.pi

    y_val_log = np.log10(y_val.copy(deep=True))
    
    return X_val_scaled, y_val_log


In [11]:
df = pd.read_csv("D:/Revi/Courses/TMLC/Projects/2. Deep Learning/4. DMSP Particle Precipitate/Dataset/AI_Ready_DMSP_Test.csv")

In [12]:
df

,Unnamed: 0,Datetimes,SC_AACGM_LAT,SC_AACGM_LTIME,ELE_TOTAL_ENERGY_FLUX,ELE_TOTAL_ENERGY_FLUX_STD,ELE_AVG_ENERGY,ELE_AVG_ENERGY_STD,ID_SC,sin_ut,...,AE_5min,AL_5min,AU_5min,SymH_5min,PC_5min,vsw_5min,vx_5min,psw_5min,borovsky_5min,newell_5min
0,429411,2003-09-14 14:21:00,52.960037,10.227748,484548200.0,0.246644,67.87744,0.364726,15.0,-0.577145,...,78.0,-17.0,61.0,5.0,0.25,397.2,-396.6,1.11,25658.352798,3826.673706
1,1420679,2011-12-04 13:46:00,69.260934,17.346047,32414300.0,0.642043,36.89621,0.801955,17.0,-0.446198,...,48.0,-56.0,-7.0,-21.0,0.77,451.9,-451.2,1.18,58399.869305,3784.324984
2,277191,2002-09-27 12:37:00,62.649816,21.031994,124487300.0,0.735324,370.75180,1.029932,14.0,-0.160743,...,172.0,-80.0,92.0,1.0,2.48,280.9,-280.4,0.37,3628.852603,2839.796580
3,793022,2005-05-25 16:18:00,55.613824,8.705475,45176510.0,1.236363,114.48180,1.751716,15.0,-0.902585,...,134.0,-78.0,56.0,0.0,0.37,387.4,-387.0,1.99,8014.554100,517.805913
4,614400,2004-08-08 18:51:00,78.081303,19.142133,73477050.0,0.464174,89.95745,0.674218,15.0,-0.975342,...,176.0,-129.0,47.0,-4.0,1.11,411.2,-410.2,0.74,13883.995956,3001.847826


In [13]:
df_test = df[feature_cols].copy()

In [14]:
df_test

,Bx_6hr,AL_6hr,vsw_6hr,psw_6hr,Bz_3hr,By_3hr,psw_1hr,By_45min,AL_45min,Bz_10min
0,-2.030000,-18.916667,427.650000,1.593333,4.401667,-0.590000,1.088333,-1.04,-10.0,-1.64
1,-1.458333,-64.166667,460.975000,1.811667,-0.093333,-2.416667,1.085000,-1.03,-32.0,-1.11
2,-0.063333,-27.916667,291.475000,1.654167,-4.473333,5.945000,0.658333,8.12,-120.0,-0.16
3,-4.309167,-31.500000,390.175000,1.367500,0.530000,3.106667,1.336667,2.58,-58.0,1.46
4,-5.596667,-63.500000,403.008333,0.605833,-0.736667,-2.423333,0.658333,-3.11,-148.0,-0.33


In [36]:
df_sample = df_test[3:4][feature_cols]

In [40]:
df_sample

,Bx_6hr,AL_6hr,vsw_6hr,psw_6hr,Bz_3hr,By_3hr,psw_1hr,By_45min,AL_45min,Bz_10min
3,-4.309167,-31.5,390.175,1.3675,0.53,3.106667,1.336667,2.58,-58.0,1.46


In [39]:
scaler_X = RobustScaler()
scaler_X = scaler_X.fit(df_sample.values)
X_val_scaled = scaler_X.transform(df_sample.values)

In [32]:
scaler_X = RobustScaler()
test = df_sample.values[0][2].reshape(-1, 1)
scaler_X = scaler_X.fit(test)
test_scaled = scaler_X.transform(test)
test,test_scaled

(array([[427.65]]), array([[0.]]))

In [41]:
df_sample = np.array(df_sample).reshape(1,-1)

In [42]:
df_sample

array([[ -4.30916667, -31.5       , 390.175     ,   1.3675    ,
          0.53      ,   3.10666667,   1.33666667,   2.58      ,
        -58.        ,   1.46      ]])

In [43]:
pred = get_prediction(df_sample,model)
pred

array([[1.]], dtype=float32)